## Using Embeddings for recommendations

In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from keras.layers import Input, concatenate
from keras import regularizers, initializers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
rating = pd.read_csv("data/ratings.csv")

movie = pd.read_csv("data/movies.csv")

In [3]:
print(rating.head())
print(rating.dtypes)
print(movie.head())
print(movie.dtypes)

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
movieId     int64
title      object
genres     object
dtype: object


###### Convert attributes to right data types

In [5]:
rating = rating.merge(movie, on='movieId', how = 'inner')

In [6]:
rating.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama


In [7]:
rating.userId = rating.userId.astype("category")

rating.movieId = rating.movieId.astype("category")

rating.genres = rating.genres.astype("category")

In [8]:
rating.dtypes

userId       category
movieId      category
rating        float64
timestamp       int64
title          object
genres       category
dtype: object

###### Check for Missing Values

In [9]:
rating.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [10]:
np.unique(rating.movieId.values)[0:100]

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  52,  53,  54,
        55,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  68,  69,
        70,  71,  72,  73,  74,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 107, 108])

In [11]:
np.unique(rating.genres.values)[0:10]

array(['(no genres listed)', 'Action', 'Action|Adventure',
       'Action|Adventure|Animation',
       'Action|Adventure|Animation|Children',
       'Action|Adventure|Animation|Children|Comedy',
       'Action|Adventure|Animation|Children|Comedy|Fantasy',
       'Action|Adventure|Animation|Children|Comedy|IMAX',
       'Action|Adventure|Animation|Children|Comedy|Romance',
       'Action|Adventure|Animation|Children|Comedy|Sci-Fi'], dtype=object)

In [12]:
userid = rating.userId.cat.codes.values

movieid = rating.movieId.cat.codes.values

genreid = rating.genres.cat.codes.values

In [13]:
np.unique(genreid)[0:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int16)

In [56]:
n_users = len(rating.userId.unique())

n_movies = len(rating.movieId.unique())

n_genres = len(rating.genres.unique())

###### Adding Cat Code Values to original dataframe

In [15]:
rating["userid_catcode"] = userid

rating["movieid_catcode"] = movieid

rating["genreid_catcode"] = genreid

In [16]:
embedding_dim = 50

#### Defining Custome Keras Metrics

In [17]:
from keras import backend as K

def mape_error(y_true, y_pred): 
    return K.mean((K.abs(y_pred - y_true)/y_true), axis=0) * 100

In [18]:
def rmse_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0)) 

In [19]:
def mse_error(y_true, y_pred):
        return K.mean(K.square(y_pred - y_true), axis=0) 

###### Split the data to Train and Test datasets

In [20]:
train_userid, test_userid, \
train_movieid, test_movieid, \
train_genreid, test_genreid, \
train_y, test_y = train_test_split(userid,
                                   movieid, 
                                   genreid,
                                   rating.rating, 
                                   test_size=0.3, random_state=2)

In [21]:
train_y.shape

(70002,)

In [23]:
train_y = train_y.values.reshape((-1,1))
test_y = test_y.values.reshape((-1,1))

In [24]:
train_y.shape
train_y[0:10]

array([[4.5],
       [3. ],
       [1. ],
       [5. ],
       [4. ],
       [2.5],
       [2. ],
       [2. ],
       [2. ],
       [5. ]])

###### Embedding UserIds

In [25]:
encoder_UserID = Sequential()
encoder_UserID.add(Embedding(n_users, embedding_dim, input_length=1,embeddings_regularizer=regularizers.l2(0.00001)))

Instructions for updating:
Colocations handled automatically by placer.


###### Embedding MovieIds

In [26]:
encoder_MovieID = Sequential()
encoder_MovieID.add(Embedding(n_movies, embedding_dim, input_length=1,embeddings_regularizer=regularizers.l2(0.00001)))

###### Embedding Genreids

In [27]:
encoder_genreID = Sequential()
encoder_genreID.add(Embedding(n_genres, embedding_dim, input_length=1,embeddings_regularizer=regularizers.l2(0.00001)))


##### Define MLP 

In [28]:
user_inp = Input(shape=(1, ))
user_mbd = Embedding(n_users, embedding_dim)(user_inp)

movie_inp = Input(shape=(1, ))
movie_mbd = Embedding(n_movies, embedding_dim)(movie_inp)

genre_inp = Input(shape=(1, ))
genre_mbd = Embedding(n_genres, embedding_dim)(genre_inp)

merged = concatenate([user_mbd, movie_mbd, genre_mbd])
fc1 = Dense(100,activation='relu')(merged)
fc2 = Dense(1)(fc1)

model = Model(inputs=[user_inp, movie_inp, genre_inp], outputs=fc2)
model.compile(optimizer='adam', loss='mse', metrics=[mse_error,rmse_error])

###### Train and fit the model

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 50)        33550       input_1[0][0]                    
____________________________________________________________________________________________

In [30]:
model.fit([train_userid,train_movieid,train_genreid], train_y.reshape(-1,1,1), epochs=100, verbose=1,batch_size=500,validation_data=([test_userid,test_movieid,test_genreid], test_y.reshape(-1,1,1)))

Instructions for updating:
Use tf.cast instead.
Train on 70002 samples, validate on 30002 samples
Epoch 1/100
70002/70002 [==============================] - 2s 35us/step - loss: 3.9240 - mse_error: 3.8986 - rmse_error: 1.6974 - val_loss: 0.8409 - val_mse_error: 0.8453 - val_rmse_error: 0.9186
Epoch 2/100
70002/70002 [==============================] - 2s 32us/step - loss: 0.7820 - mse_error: 0.7813 - rmse_error: 0.8832 - val_loss: 0.8052 - val_mse_error: 0.8140 - val_rmse_error: 0.9009
Epoch 3/100
70002/70002 [==============================] - 2s 33us/step - loss: 0.7339 - mse_error: 0.7331 - rmse_error: 0.8557 - val_loss: 0.8113 - val_mse_error: 0.8165 - val_rmse_error: 0.9027
Epoch 4/100
70002/70002 [==============================] - 2s 31us/step - loss: 0.7207 - mse_error: 0.7197 - rmse_error: 0.8474 - val_loss: 0.8105 - val_mse_error: 0.8185 - val_rmse_error: 0.9035
Epoch 5/100
70002/70002 [==============================] - 2s 30us/step - loss: 0.7105 - mse_error: 0.7217 - rmse_erro

70002/70002 [==============================] - 2s 33us/step - loss: 0.4535 - mse_error: 0.4510 - rmse_error: 0.6701 - val_loss: 0.9152 - val_mse_error: 0.9248 - val_rmse_error: 0.9603
Epoch 42/100
70002/70002 [==============================] - 2s 32us/step - loss: 0.4382 - mse_error: 0.4363 - rmse_error: 0.6594 - val_loss: 0.9133 - val_mse_error: 0.9243 - val_rmse_error: 0.9599
Epoch 43/100
70002/70002 [==============================] - 2s 31us/step - loss: 0.4295 - mse_error: 0.4289 - rmse_error: 0.6541 - val_loss: 0.9266 - val_mse_error: 0.9369 - val_rmse_error: 0.9664
Epoch 44/100
70002/70002 [==============================] - 2s 34us/step - loss: 0.4215 - mse_error: 0.4196 - rmse_error: 0.6466 - val_loss: 0.9361 - val_mse_error: 0.9508 - val_rmse_error: 0.9731
Epoch 45/100
70002/70002 [==============================] - 2s 33us/step - loss: 0.4128 - mse_error: 0.4102 - rmse_error: 0.6388 - val_loss: 0.9293 - val_mse_error: 0.9371 - val_rmse_error: 0.9669
Epoch 46/100
70002/70002 [==

Epoch 83/100
70002/70002 [==============================] - 2s 30us/step - loss: 0.1503 - mse_error: 0.1506 - rmse_error: 0.3873 - val_loss: 1.2471 - val_mse_error: 1.2479 - val_rmse_error: 1.1160
Epoch 84/100
70002/70002 [==============================] - 2s 30us/step - loss: 0.1520 - mse_error: 0.1511 - rmse_error: 0.3873 - val_loss: 1.2348 - val_mse_error: 1.2397 - val_rmse_error: 1.1122
Epoch 85/100
70002/70002 [==============================] - 2s 29us/step - loss: 0.1407 - mse_error: 0.1399 - rmse_error: 0.3731 - val_loss: 1.2359 - val_mse_error: 1.2423 - val_rmse_error: 1.1133
Epoch 86/100
70002/70002 [==============================] - 2s 30us/step - loss: 0.1386 - mse_error: 0.1394 - rmse_error: 0.3726 - val_loss: 1.2746 - val_mse_error: 1.2711 - val_rmse_error: 1.1262
Epoch 87/100
70002/70002 [==============================] - 2s 32us/step - loss: 0.1421 - mse_error: 0.1415 - rmse_error: 0.3754 - val_loss: 1.2512 - val_mse_error: 1.2579 - val_rmse_error: 1.1203
Epoch 88/100
70

###### Evaluating Scores Manually

In [32]:
scores = model.evaluate([test_userid,test_movieid,test_genreid], test_y.reshape(-1,1,1))

print(scores)

30002/30002 [==============================] - 1s 25us/step
[1.3159168274925037, 1.3158081769943237, 1.1330957412719727]


In [33]:
test_pred = model.predict([test_userid,test_movieid,test_genreid])

In [34]:
mse_score = np.mean(np.square(test_pred.reshape(-1,1) - test_y))

print("mse_score:",mse_score)

mse_score: 1.3159168251902307


###### Calculate MSE and RMSE Manually

In [35]:
rmse_score = np.sqrt(np.mean(np.square(test_pred.reshape(-1,1) - test_y)))

print("rmse_score:",rmse_score)

rmse_score: 1.1471341792441854


## Developing Recommendations

###### Extracting/Predicting user embeddings

In [36]:
encoder_UserID = Model(user_inp, user_mbd)

encoder_MovieID = Model(movie_inp, movie_mbd)

In [37]:
users_unique = np.unique(userid)

users_embeddings = encoder_UserID.predict(users_unique)

In [38]:
users_unique[0:3]

users_embeddings[0:3]

array([[[ 0.00937831,  0.09306882,  0.02798257, -0.11451815,
         -0.17154603,  0.10121604,  0.00219925, -0.09427488,
          0.08729226, -0.01271356,  0.0480085 , -0.06021031,
          0.0686344 ,  0.06600476, -0.04319668, -0.02193855,
         -0.17557213, -0.04872634,  0.11772239, -0.02861193,
         -0.08993847, -0.07496922,  0.04350245,  0.06192971,
         -0.09684246,  0.0544726 , -0.06157613,  0.00843827,
         -0.06087888,  0.28190553,  0.13663387,  0.07415251,
         -0.07918856, -0.07101006,  0.09267204, -0.02502137,
         -0.10163342, -0.08868191,  0.04871479,  0.18856154,
         -0.24165097, -0.06425928,  0.17878199,  0.14739783,
         -0.01084118,  0.04854346,  0.03096857,  0.01866068,
          0.07438418, -0.22130443]],

       [[-0.15971145,  0.06707465, -0.16358429, -0.05157582,
         -0.20874792,  0.0806102 , -0.05452153,  0.05546542,
          0.09328122,  0.14510234,  0.21460621, -0.01733369,
         -0.1257984 , -0.1534178 , -0.2128454 ,

In [39]:
users_embeddings = pd.DataFrame(users_embeddings.reshape(-1,50))

users_embeddings["userid_catcode"] = users_unique

In [40]:
users_embeddings[0:3]

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,userid_catcode
0,0.009378,0.093069,0.027983,-0.114518,-0.171546,0.101216,0.002199,-0.094275,0.087292,-0.012714,...,-0.064259,0.178782,0.147398,-0.010841,0.048543,0.030969,0.018661,0.074384,-0.221304,0
1,-0.159711,0.067075,-0.163584,-0.051576,-0.208748,0.080610,-0.054522,0.055465,0.093281,0.145102,...,-0.108943,0.063410,-0.145408,0.069672,0.078602,-0.193921,-0.284708,0.306774,-0.124459,1
2,-0.152890,0.104519,0.004225,-0.137203,-0.086017,0.041962,-0.200898,0.235378,-0.216183,0.023248,...,-0.251649,-0.021319,-0.009064,-0.077345,-0.040086,-0.040156,0.018226,0.088419,0.001760,2


In [41]:
movies_unique = np.unique(movieid)

movies_embeddings = encoder_MovieID.predict(movies_unique)

In [42]:
movies_embeddings = pd.DataFrame(movies_embeddings.reshape(-1,50))

movies_embeddings["movieid_catcode"] = movies_unique

##### Identify nearest neighbours for a movie based on k-Nearest Neighbours Algorithm

In [43]:
from sklearn.neighbors import NearestNeighbors

In [44]:
nbrs = NearestNeighbors(n_neighbors = 10, algorithm='auto').fit(movies_embeddings.drop(["movieid_catcode"],axis=1))

movie_distances, movie_nbrs = nbrs.kneighbors(movies_embeddings.drop(["movieid_catcode"], axis=1))

movie_nbrs = pd.DataFrame(movie_nbrs)

movie_nbrs.columns = ["movieid_catcode","NN1","NN2","NN3","NN4","NN5","NN6","NN7","NN8","NN9"]

###### Define Recommender Function

In [45]:
def recommender(rating,movie_nbrs,model,uid,n):
    
    # Mapping the userid to its Cat Code
    uid_catcode=rating[rating.userId==uid]["userid_catcode"].unique()
    
    #Subsetting the records of the userid
    movies=(rating[rating.userid_catcode==uid_catcode[0]])
    
    #Sorting the user movies based on given ratings
    if len(movies)>=n :
        movies=movies.sort_values("rating",ascending=False)
        movies=list((movies[0:n]["movieid_catcode"]))
    else :
        movies=list(movies["movieid_catcode"])
    
   
    Movie_nbrs=movie_nbrs[movie_nbrs.movieid_catcode.isin(movies)]
    Movie_nbrs=Movie_nbrs.drop("movieid_catcode",axis=1)
    Movie_nbrs=pd.DataFrame(np.unique(Movie_nbrs.values.flatten()))
    Movie_nbrs.columns=["movieid_catcode"]
    Movie_nbrs=Movie_nbrs[~(Movie_nbrs["movieid_catcode"].isin(list(rating[rating.userid_catcode==uid_catcode[0]]["movieid_catcode"])))]
    genre_nbrs=rating[rating.movieid_catcode.isin(Movie_nbrs.movieid_catcode)]["genreid_catcode"]
    genre_nbrs.columns=["genre_id"]
    
    
    if len(Movie_nbrs)>0:
        usid=np.repeat(uid_catcode[0], len(Movie_nbrs))
        Movie_nbrs=Movie_nbrs.movieid_catcode.values
        genre_nbrs = genre_nbrs.values
        test_pred = model.predict([usid,Movie_nbrs,genre_nbrs]).reshape(-1)
        
        r_Df=pd.DataFrame({"movieid_catcode":Movie_nbrs})
        r_Df["rating"]=test_pred
        
        Movie_catcodes=rating[["movieId","movieid_catcode","title"]]
        Movie_catcodes=Movie_catcodes.drop_duplicates()
        r_Df=pd.merge(r_Df,Movie_catcodes,how="left",on=["movieid_catcode"])
        r_Df=r_Df.sort_values("rating",ascending=False)
        r_Df=r_Df.iloc[0:n]
        r_Df["userId"] = uid
        return(r_Df)
    else :
        print("Try with another n value")
        r_Df=None
        return(r_Df)

###### Recommend movie for a user

In [46]:
uid = 605

n = 5

model = model

rec = recommender(rating, movie_nbrs, model, uid, n)

In [47]:
rec

,movieid_catcode,rating,movieId,title,userId
7,2427,5.049459,3021,"Funhouse, The (1981)",605
23,4866,5.005537,6918,"Unvanquished, The (Aparajito) (1957)",605
31,7032,4.925920,62376,City of Ember (2008),605
41,8687,4.895577,116897,Wild Tales (2014),605
36,8157,4.868207,97957,Excision (2012),605
